# Plotting time-series in R using pluto

A getting started guide

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)

options("scipen"=999)
options(stringsAsFactors = FALSE)
source("config.R")

indices<-Indices()

Let's pick a long time-series. This will allow us to play with different aspects of plotting...

In [ ]:
cutOff <- Sys.Date() - 10
trIndices <- indices$NseTimeSeries() %>%
    group_by(NAME) %>%
    summarize(ST = min(TIME_STAMP), ET = max(TIME_STAMP)) %>%
    filter(ST <= '2005-01-01' && ET >= cutOff && NAME %like% '% TR') %>%
    arrange(ST)

trIndices %>% print(n=Inf)

NIFTY 500 TR makes a good candidate. It is a broad based index the represents the top-500 stocks by free-float market cap listed on the NSE...

In [ ]:
indexName <- "NIFTY 500 TR"
trDf <- data.frame(indices$NseTimeSeries() %>%
    filter(NAME == indexName) %>%
    select(TIME_STAMP, HIGH, LOW, OPEN, CLOSE))
trXts<-xts(trDf[,-1], trDf[,1])

print(head(trXts))
print(tail(trXts))

Note: the series only has closing prices. 

In [ ]:
#setup the data for ggplot
firstDate <- first(index(trXts))
lastDate <- last(index(trXts))
xAxisTicks <- seq(from=firstDate, to=lastDate, length.out=10)

toPlotDf <- data.frame(trXts[, 'CLOSE'])
names(toPlotDf) <- c('C')
toPlotDf$C <- as.numeric(toPlotDf$C)
toPlotDf$T <- index(trXts)

#options(repr.plot.width=16, repr.plot.height=8)

### plot the series as-is

In [ ]:
ggplot(toPlotDf, aes(x=T, y=C))
    #theme_economist() 
    #scale_x_date(breaks = xAxisTicks) +
    #labs(x='', y='price', color='', title=sprintf("%s", indexName), subtitle=sprintf("[%s:%s]", firstDate, lastDate)) +
    #annotate("text", x=lastDate, y=min(toPlotDf$C, na.rm=T), label = "@StockViz", hjust=1.1, vjust=-1.1, col="white", cex=6, fontface = "bold", alpha = 0.8)